<a href="https://colab.research.google.com/github/DaniyalK03/Options-Vol.Surface-and-Arbitrage-Calculator/blob/main/Option_Pricer_and_Volatility_Surface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################################################################

# Importing standard backend and libraries

#Import standard libraries and imageio to read in the images
import imageio.v3 as imageio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import math

# Importing TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

## BS Calculator

In [ ]:
from scipy.stats import norm
import numpy as np

def BS_option(S0, K, T, r, sigma, option_type = ""):
  """

  This function prices European Call & Put options using the Black-Scholes Model given

  S0 = Price of the stock today
  K = Strike Price
  T = Time to Maturity in Years
  r = Risk-free interest rate as Percent
  sigma = Volatility

  The result is the price of the option today

  """
  r = r/100
  d1 = (np.log(S0/K) + (r + (sigma**2)/2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - (sigma * np.sqrt(T))
  if option_type == "Call":
    #return print(f"Call Price = {S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2):0.2f}")
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
  elif option_type == "Put":
    #return print(f"Put Price = {K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1):0.2f}")
    return K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
  else:
    print("ERROR, Please ensure you have specified an option with proper capitalisation")

In [ ]:
#BS_option(100, 100, 1, 8, 0.1, "Call")
#BS_option(100, 100, 1, 8, 0.1, "Put")
#BS_option(17.99, 15, 1, 5, 0.012, "Call")
BS_option(116.37, 130, 1, 5, 0.1, "Call")
BS_option(116.37, 130, 1, 5, 0.1, "Put")

Call Price = 2.00
Put Price = 9.29


## Arbitrage Identification and Profit Strategising

In [ ]:
def Arbitrage(S0, K, T, r, call_price, put_price):

  call_price = float(call_price)
  put_price = float(put_price)

  r = r/100
  r = float(r)
  S0 = float(S0)
  K = float(K)
  T = float(T)

  lhs = put_price + S0
  rhs = call_price + K*np.exp(-r*T)

  avg_value = (lhs + rhs) / 2
  tolerance = 1e-5 * avg_value

  print(f"Tolerance: {tolerance} \n")


  if lhs < rhs - tolerance:
    arbitrage_profit = rhs - lhs

    # Show profit with precision based on the arbitrage size
    if round(arbitrage_profit, 2) > 0:
      print(f"Put-Call Parity Theorem states Put is underpriced relative to the Call.")
      print(f"Final Arbitrage Profit = {arbitrage_profit:0.2f}")
      print(f"\n Buy Put & Buy Stock \n Sell Call \n Invest cash difference into bank account to earn compound interest. \n Exercise option at {T} years. \n")
    else:
      print(f"Put-Call Parity Theorem states the options are priced fairly and there is no arbitrage.\n")


  elif lhs > rhs + tolerance:
    arbitrage_profit = lhs - rhs

    # Show profit with precision based on the arbitrage size
    if round(arbitrage_profit, 2) > 0:
      print(f"Put-Call Parity Theorem states Put is overpriced relative to the Call.")
      print(f"Final Arbitrage Profit = {arbitrage_profit:0.2f}")
      print(f"\n Sell Put & Sell Stock \n Buy Call \n Invest cash difference into bank account to earn compound interest. \n Exercise option at {T} years. \n")
    else:
      print(f"Put-Call Parity Theorem states the options are priced fairly and there is no arbitrage.\n")


  else:
    print("Put-Call Parity Theorem states the options are priced fairly and there is no arbitrage.\n")

In [ ]:
#Arbitrage(100, 100, 1, 8, 8.84, 1.15)
#Arbitrage(100, 100, 1, 8, 10.50, 0.50)
Arbitrage(100, 100, 1, 8, 7.00, 4.50)
Arbitrage(116.37, 130, 1, 5, 2, 9.29)

Tolerance: 0.001019058173193318 

Put-Call Parity Theorem states Put is overpriced relative to the Call.
Final Arbitrage Profit = 5.19

 Sell Put & Sell Stock 
 Buy Call 
 Invest cash difference into bank account to earn compound interest. 
 Exercise option at 1.0 years. 

Tolerance: 0.0012565991259254642 

Put-Call Parity Theorem states the options are priced fairly and there is no arbitrage.



## Implementing Volatility Surface

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.interpolate import griddata
import plotly.graph_objects as go


In [ ]:
##################################################################################################################
def implied_vol(price, S, K, T, r, option_type="Call", tol=1e-8, max_iter=100):
    """Return Black-Scholes implied vol; np.nan if out of bounds or no root in [1e-6, 5]."""
    if T <= 0:
        return np.nan

    intrinsic = max(0.0, S - K) if option_type == "Call" else max(0.0, K - S)
    discount_K = K * np.exp(-r * T)
    if option_type == "Call":
        upper_bound = S
        lower_bound = max(0.0, S - discount_K)
    else:
        upper_bound = discount_K
        lower_bound = max(0.0, discount_K - S)

    if not (lower_bound - 1e-12 <= price <= upper_bound + 1e-12):
        return np.nan

    f = lambda sigma: BS_option(S, K, T, r, sigma, option_type) - price
    try:
        return float(brentq(f, 1e-6, 5.0, xtol=tol, maxiter=max_iter))
    except Exception:
        return np.nan

### Legacy Code for single surface (IGNORE)

In [ ]:
"""
# -------------------------------
# Surface builder (interpolates IV across K,T)
# -------------------------------

def build_iv_surface(option_rows, method="linear", fill_value=np.nan):
    Ks, Ts, ivs = [], [], []
    for row in option_rows:
        S = float(row["S"])
        K = float(row["K"])
        T = float(row["T"])
        r = float(row["r"])
        price = row["price"]
        #opt_type = row.get("option_type", "Call").lower()
        opt_type = str(row.get("option_type", "Call")).strip().capitalize()

        # --- Skip invalid rows ---
        if price is None or (isinstance(price, float) and np.isnan(price)):
            continue

        try:
            price = float(price)
        except Exception:
            continue

        iv = implied_vol(price, S, K, T, r, opt_type)
        if not np.isnan(iv):
            Ks.append(K)
            Ts.append(T)
            ivs.append(iv)

    if len(ivs) == 0:
        raise ValueError("No valid implied vols from the provided data.")

    points = np.column_stack([Ks, Ts])
    iv = np.array(ivs)

    def iv_interp(query_K, query_T):
        q = np.column_stack([np.ravel(query_K), np.ravel(query_T)])
        vals = griddata(points, iv, q, method=method, fill_value=fill_value)
        return vals.reshape(np.shape(query_K))

    return points, iv, iv_interp


import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_iv_surfaces(df_quotes, iv_interp_func, nK=40, nT=30):
    # Build grid
    K_min, K_max = df_quotes["K"].min(), df_quotes["K"].max()
    T_min, T_max = df_quotes["T"].min(), df_quotes["T"].max()

    K_lin = np.linspace(K_min, K_max, nK)
    T_lin = np.linspace(T_min, T_max, nT)
    KK, TT = np.meshgrid(K_lin, T_lin, indexing="xy")

    # If only one option_type -> single surface
    opt_types = df_quotes["option_type"].unique()

    if len(opt_types) == 1:
        opt = opt_types[0]
        IV_grid = iv_interp_func(KK, TT)
        fig = go.Figure(data=[go.Surface(x=KK, y=TT, z=IV_grid, colorbar=dict(title="IV"))])
        fig.update_layout(
            title=f"Implied Volatility Surface ({opt})",
            scene=dict(
                xaxis_title="Strike (K)",
                yaxis_title="Time to Maturity (T, years)",
                zaxis_title="Implied Vol"
            ),
            height=600
        )
        fig.show()

    # If both Calls & Puts -> two subplots
    elif set(opt_types) == {"Call","Put"} or set(opt_types) == {"Put","Call"}:
        fig = make_subplots(rows=1, cols=2,
                            specs=[[{'type': 'surface'}, {'type': 'surface'}]],
                            subplot_titles=("Call IV Surface", "Put IV Surface"))

        for i, opt in enumerate(["Call","Put"], start=1):
            df_sub = df_quotes[df_quotes["option_type"] == opt]
            # Rebuild surface for each type separately
            _, _, iv_interp = build_iv_surface(df_sub.to_dict("records"), method="linear")
            IV_grid = iv_interp(KK, TT)
            fig.add_trace(
                go.Surface(x=KK, y=TT, z=IV_grid, showscale=True, colorbar=dict(title="IV")),
                row=1, col=i
            )

        fig.update_layout(
            title="Implied Volatility Surfaces (Calls vs Puts)",
            scene=dict(xaxis_title="Strike", yaxis_title="Maturity", zaxis_title="IV"),
            height=600
        )
        fig.show()


# -------------------------------
# Synthetic dataset for quick testing
# -------------------------------

def synthetic_quotes(S0=100.0, r=0.01, base_vol=0.2, skew=-0.2, term_structure=0.1,
                     K_grid=None, T_grid=None, option_types=("Call","Put")):
    if K_grid is None:
        K_grid = np.linspace(60, 140, 25)
    if T_grid is None:
        T_grid = np.linspace(0.05, 2.0, 15)

    rows = []
    for T in T_grid:
        for K in K_grid:
            m = (K / S0) - 1.0
            iv = base_vol * (1 + skew * m) * (1 + term_structure * np.sqrt(T))
            iv = float(np.clip(iv, 0.01, 2.0))
            price = BS_option(S0, K, T, r, iv, option_types)
            rows.append({"S": S0, "K": K, "T": T, "r": r, "price": price, "option_type": option_types})
    return pd.DataFrame(rows)

def synthetic_quotes(S0=100.0, r=0.01, base_vol=0.2, skew=-0.2, term_structure=0.1,
                     K_grid=None, T_grid=None, option_types=("Call", "Put")):
    if K_grid is None:
        K_grid = np.linspace(60, 140, 25)
    if T_grid is None:
        T_grid = np.linspace(0.05, 2.0, 15)

    rows = []
    for T in T_grid:
        for K in K_grid:
            m = (K / S0) - 1.0
            iv = base_vol * (1 + skew * m) * (1 + term_structure * np.sqrt(T))
            iv = float(np.clip(iv, 0.01, 2.0))
            # loop over each option type individually
            for opt_type in option_types:
                price = BS_option(S0, K, T, r, iv, opt_type)
                rows.append({
                    "S": S0, "K": K, "T": T, "r": r,
                    "price": price, "option_type": opt_type
                })
    return pd.DataFrame(rows)
    """

### Implied Volatility Calculation

In [ ]:
def build_iv_surface(option_rows, method="linear", fill_value=np.nan):
    Ks, Ts, ivs = [], [], []
    for row in option_rows:
        S = float(row["S"]); K = float(row["K"]); T = float(row["T"]); r = float(row["r"])
        price = row["price"]
        opt_type = str(row.get("option_type", "Call")).strip().capitalize()   # <- normalize

        if price is None or (isinstance(price, float) and np.isnan(price)):
            continue
        price = float(price)

        iv = implied_vol(price, S, K, T, r, opt_type)
        if not np.isnan(iv):
            Ks.append(K); Ts.append(T); ivs.append(iv)
    if not ivs:
        raise ValueError("No valid implied vols from the provided data.")
    points = np.column_stack([Ks, Ts]); iv = np.array(ivs)

    def iv_interp(Kq, Tq):
        q = np.column_stack([np.ravel(Kq), np.ravel(Tq)])
        vals = griddata(points, iv, q, method=method, fill_value=fill_value)
        return vals.reshape(np.shape(Kq))
    return points, iv, iv_interp


### Synthetic Datasets

In [ ]:
def synthetic_quotes(S0=100.0, r=0.01, base_vol=0.2, skew=-0.2, term_structure=0.1,
                     K_grid=None, T_grid=None, option_types=("Call", "Put")):
    import numpy as np
    import pandas as pd

    # Accept "Call" or ["Call","Put"]
    if isinstance(option_types, str):
        option_types = [option_types]

    if K_grid is None:
        K_grid = np.linspace(0.6 * S0, 1.4 * S0, 25)
    if T_grid is None:
        T_grid = np.linspace(0.05, 2.0, 15)

    rows = []  # <-- initialize list before appending

    for T in T_grid:
        for K in K_grid:
            # moneyness relative to spot
            m = (K / S0) - 1.0
            iv = float(np.clip(base_vol * (1 + skew * m) * (1 + term_structure * np.sqrt(T)), 0.01, 2.0))
            for opt_type in option_types:
                price = BS_option(S0, K, T, r, iv, opt_type)  # your fixed numeric-returning BS_option
                rows.append({
                    "S": float(S0), "K": float(K), "T": float(T), "r": float(r),
                    "price": float(price), "option_type": str(opt_type)
                })

    return pd.DataFrame(rows)



### Surface Plotter

In [ ]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_iv_surfaces(df_quotes, nK=40, nT=30):
    K_min, K_max = df_quotes["K"].min(), df_quotes["K"].max()
    T_min, T_max = df_quotes["T"].min(), df_quotes["T"].max()
    K_lin = np.linspace(K_min, K_max, nK)
    T_lin = np.linspace(T_min, T_max, nT)
    KK, TT = np.meshgrid(K_lin, T_lin, indexing="xy")

    types = sorted(df_quotes["option_type"].astype(str).str.strip().str.capitalize().unique())
    if types == ["Call", "Put"]:
        # build surfaces separately to respect each cross-section’s quotes
        _, _, iv_call = build_iv_surface(df_quotes[df_quotes.option_type.str.contains("Call", case=False)].to_dict("records"))
        _, _, iv_put  = build_iv_surface(df_quotes[df_quotes.option_type.str.contains("Put",  case=False)].to_dict("records"))
        Zc = iv_call(KK, TT); Zp = iv_put(KK, TT)
        zmin = float(np.nanmin([Zc, Zp])); zmax = float(np.nanmax([Zc, Zp]))

        fig = make_subplots(rows=1, cols=2,
                            specs=[[{"type":"surface"},{"type":"surface"}]],
                            subplot_titles=("Call IV Surface","Put IV Surface"))

        fig.add_trace(go.Surface(x=KK, y=TT, z=Zc, showscale=True, colorbar=dict(title="IV"),
                                 cmin=zmin, cmax=zmax), row=1, col=1)
        fig.add_trace(go.Surface(x=KK, y=TT, z=Zp, showscale=True, colorbar=dict(title="IV"),
                                 cmin=zmin, cmax=zmax), row=1, col=2)

        # label BOTH 3D scenes
        fig.update_layout(
            scene = dict(xaxis_title="Strike (K)", yaxis_title="Maturity (T, years)", zaxis_title="Implied Vol"),
            scene2= dict(xaxis_title="Strike (K)", yaxis_title="Maturity (T, years)", zaxis_title="Implied Vol"),
            height=600, title="Implied Volatility Surfaces (Calls vs Puts)"
        )
        cam = dict(eye=dict(x=1.6, y=1.6, z=0.8))
        fig.update_layout(scene_camera=cam, scene2_camera=cam)
        fig.show()
    else:
        # single surface fallback
        _, _, iv_interp = build_iv_surface(df_quotes.to_dict("records"))
        Z = iv_interp(KK, TT)
        fig = go.Figure(go.Surface(x=KK, y=TT, z=Z, colorbar=dict(title="IV")))
        fig.update_layout(scene=dict(xaxis_title="Strike (K)", yaxis_title="Maturity (T, years)", zaxis_title="Implied Vol"),
                          height=600, title=f"Implied Volatility Surface ({types[0]})")
        fig.show()


In [ ]:

# Synthetic example (replace with your real df)
df_quotes = synthetic_quotes(S0=100.0, r=0.01, base_vol=0.20, skew=-0.25, term_structure=0.10, option_types=["Call", "Put"])
df_quotes.head()


,S,K,T,r,price,option_type
0,100.0,60.000000,0.05,0.01,4.000030e+01,Call
1,100.0,60.000000,0.05,0.01,5.803936e-25,Put
2,100.0,63.333333,0.05,0.01,3.666698e+01,Call
3,100.0,63.333333,0.05,0.01,1.193068e-20,Put
4,100.0,66.666667,0.05,0.01,3.333367e+01,Call


### Legacy Code for single surface (IGNORE)

In [ ]:
"""
# Compute implied vols and build the interpolator
points, iv_raw, iv_interp = build_iv_surface(df_quotes.to_dict("records"), method="linear", fill_value=np.nan)
print(f"Computed {np.isfinite(iv_raw).sum()} implied volatilities.")
"""

Computed 685 implied volatilities.


In [ ]:
"""
# Build a regular grid to visualize the surface
K_min, K_max = df_quotes["K"].min(), df_quotes["K"].max()
T_min, T_max = df_quotes["T"].min(), df_quotes["T"].max()

K_lin = np.linspace(K_min, K_max, 50)
T_lin = np.linspace(T_min, T_max, 30)
KK, TT = np.meshgrid(K_lin, T_lin, indexing="xy")

IV_grid = iv_interp(KK, TT)
"""

In [ ]:
"""
# 3D surface
fig = go.Figure(data=[go.Surface(x=KK, y=TT, z=IV_grid, colorbar=dict(title="IV"))])
fig.update_layout(
    title="Implied Volatility Surface",
    scene=dict(xaxis_title="Strike (K)", yaxis_title="Time to Maturity (T, years)", zaxis_title="Implied Vol"),
    height=600
)
fig.show()

"""

### Visual Comparison

In [ ]:
# If df_quotes contains both Calls and Puts
plot_iv_surfaces(df_quotes)


## Exploring American Options

In [ ]:
def binomial_tree(S, K, T, r, sigma, n=100, option_type='call'):
    dt = T/n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1/u
    p = (np.exp(r * dt) - d) / (u - d)
    # Initialize stock price tree
    stock_tree = np.zeros((n+1, n+1))
    option_tree = np.zeros((n+1, n+1))

    # Stock prices at maturity
    for i in range(n+1):
        stock_tree[i, n] = S * (u ** (n-i)) * (d ** i)

    # Option payoffs at maturity
    if option_type == 'call':
        option_tree[:, n] = np.maximum(0, stock_tree[:, n] - K)
    else:
        option_tree[:, n] = np.maximum(0, K - stock_tree[:, n])

    # Backward induction
    for j in range(n-1, -1, -1):
        for i in range(j+1):
            option_tree[i, j] = np.exp(-r * dt) * (p * option_tree[i, j+1] + (1-p) * option_tree[i+1, j+1])
    return option_tree[0, 0]